In [1]:
import numpy as np
import pandas as pd

In [2]:
import tensorflow as tf
import tensorflow.keras as k
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense, Input, Conv2D, MaxPool2D, Flatten, Reshape

In [3]:
import silence_tensorflow

In [4]:
from keras_bed_sequence import BedSequence
from keras_mixed_sequence import MixedSequence

# Data

In [5]:
df = pd.read_csv("./mendelian_snv.csv.gz")

In [6]:
chromosomes = df.chrom.unique()

In [7]:
train_bed = df.head(int(1e5))

In [8]:
X = BedSequence(
    assembly="hg19",
    bed=train_bed,
    batch_size=128
)

In [9]:
y = train_bed.labels.values
y = y.astype(float)

In [10]:
X[0].shape

(128, 500, 4)

In [11]:
y.shape

(100000,)

In [12]:
mixed = MixedSequence(X, y, 128)

In [13]:
mixed.on_epoch_end()

# Model

In [14]:
def build_model(x, filters, kernels, pools, dense):
    # Convolutional part
    for  _filter, _kernel, _pool in zip(filters, kernels, pools):
        x = Conv2D(_filter, _kernel, activation="relu", padding="same")(x)
        x = MaxPool2D(_pool)(x)

    x = Flatten()(x)
    
    # Dense part
    for _dense in dense:
        x = Dense(_dense, activation="relu")(x)
        
    x = Dense(1, activation="sigmoid")(x)
    return x

In [15]:
filters = [128, 64, 64, 32]
kernels = [(9, 4), (6, 2), (3, 1), (3, 1)]
pools   = [(3, 1), (2, 1), (2, 2), (2, 2)]
dense   = [256, 64, 32]

In [16]:
i = Input(
    shape=(500, 4)
)
x = Reshape((500, 4, 1))(i)
classifier = Model(
    inputs=i,
    outputs=build_model(
        x,
        filters,
        kernels,
        pools,
        dense
    )
)
classifier.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 500, 4)]          0         
_________________________________________________________________
reshape (Reshape)            (None, 500, 4, 1)         0         
_________________________________________________________________
conv2d (Conv2D)              (None, 500, 4, 128)       4736      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 166, 4, 128)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 166, 4, 64)        98368     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 83, 4, 64)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 83, 4, 64)         12352 

In [17]:
def balanced_binary_crossentropy(weight=1.0):
    def loss(y_true, y_pred):
        mask = y_true==1
        nmask = ~mask
        l1 = tf.cond(
            k.backend.any(mask),
            true_fn=lambda : k.backend.categorical_crossentropy(y_true[mask], y_pred[mask]),
            false_fn=lambda : 0.0,
            name=None
        )
        l0 = tf.cond(
            k.backend.any(nmask),
            true_fn=lambda : k.backend.categorical_crossentropy(y_true[nmask], y_pred[nmask]),
            false_fn=lambda : 0.0,
            name=None
        )
            
        return l0 + weight * l1
    return loss

In [18]:
classifier.compile(
    optimizer="nadam",
    loss=balanced_binary_crossentropy(2.0)
)

In [19]:
histoty = classifier.fit_generator(
    generator=mixed,
    steps_per_epoch=mixed.steps_per_epoch // 5,
    epochs=100,
    verbose=1,
    use_multiprocessing=False,
    shuffle=True
).history

Train for 156 steps
Epoch 1/100
156/156 [==============================] - 70s 450ms/step - loss: 0.2622
Epoch 2/100
156/156 [==============================] - 72s 460ms/step - loss: 0.3622
Epoch 3/100
156/156 [==============================] - 74s 473ms/step - loss: 0.2978
Epoch 4/100
156/156 [==============================] - 72s 462ms/step - loss: 0.3156
Epoch 5/100
156/156 [==============================] - 73s 467ms/step - loss: 0.2089
Epoch 6/100
156/156 [==============================] - 72s 460ms/step - loss: 0.3866
Epoch 7/100
156/156 [==============================] - 74s 472ms/step - loss: 0.2757
Epoch 8/100
156/156 [==============================] - 73s 471ms/step - loss: 0.2378
Epoch 9/100
156/156 [==============================] - 72s 462ms/step - loss: 0.2512
Epoch 10/100
156/156 [==============================] - 74s 476ms/step - loss: 0.1667
Epoch 11/100
156/156 [==============================] - 74s 472ms/step - loss: 0.3112
Epoch 12/100
156/156 [=====================

KeyboardInterrupt: 